In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard

import os
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import time


training_path = "fruits-360_dataset/Training"
test_path = "fruits-360_dataset/Test"

try:
    STATS = np.load("stats.npy", allow_pickle=True)
except FileNotFoundError as fnf:
    print("Not found stats file.")
    STATS = []

# Parameters    
GRAY_SCALE = False
FRUITS = os.listdir(training_path)
random.shuffle(FRUITS)

train_load = 0.1
test_load = 0.3



In [ ]:
def load_data(directory_path, load_factor=None):
    data = []
    labels = []
               
            
    for fruit_name in FRUITS:
        class_num = FRUITS.index(fruit_name)                
        
        path = os.path.join(directory_path, fruit_name)
        
        for img in os.listdir(path):
            if load_factor and np.random.random() > load_factor:  # skip image
                continue
                
            img_path = os.path.join(path, img)    
            if GRAY_SCALE:
                image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            else:
                image = cv2.imread(img_path)
                image = image[:, :, [2, 1, 0]]
                
            image = image / 255.0
            image = np.array(image, dtype=np.single)  # Reduce precision and memory consumption
                             
            data.append([image, class_num])

    random.shuffle(data)
        
    X = []
    y = []
    
    
    for image, label in data:
        X.append(image)
        y.append(label)
    
    X = np.array(X)
    y = np.array(y)
    
    if GRAY_SCALE:
        print("Reshaping gray scale")
        X = X.reshape(-1, X.shape[1], X.shape[2], 1)
        
    return X, y      

In [ ]:
X_training, y_training = load_data(training_path, load_factor=train_load)
print("Created training array")    
print(f"X shape: {X_training.shape}")
print(f"y shape: {y_training.shape}")

In [ ]:
X_test, y_test = load_data(test_path, load_factor=test_load)

print("Created test arrays")    
print(f"X shape: {X_test.shape}")
print(f"y shape: {y_test.shape}")

In [ ]:
class AfterTwoEpochStop(tf.keras.callbacks.Callback):
    def __init__(self, acc_threshold, loss_threshold):
#         super(AfterTwoEpochStop, self).__init__()
        self.acc_threshold = acc_threshold
        self.loss_threshold = loss_threshold
        self.checked = False
        print("Init")

    def on_epoch_end(self, epoch, logs=None): 
        acc = logs["accuracy"]   
        loss = logs["loss"]
        if acc >= self.acc_threshold and loss <= self.loss_threshold:
            if self.checked:
                self.model.stop_training = True
            else:
                self.checked = True
        else:
            self.checked = False

stop = AfterTwoEpochStop(acc_threshold=0.98, loss_threshold=0.05)                

In [ ]:
# Limit gpu memory usage

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = False
config.gpu_options.per_process_gpu_memory_fraction = 0.4
session = tf.compat.v1.Session(config=config)


In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Conv3D, MaxPooling2D, MaxPooling3D, Activation, Dropout

In [ ]:
dense_layers = [2]
dense_size = [32, 64]
conv_layers = [1, 2, 3]
conv_size = [32, 64]
conv_shape = [2, 5]

pic_shape = X_training.shape[1:]
label_count = len(FRUITS)


run_num = 0
total = len(dense_layers)*len(dense_size)*len(conv_layers)*len(conv_size)*len(conv_shape)
for dl in dense_layers:
    for ds in dense_size:
        for cl in conv_layers:
            for cs in conv_size:                
                for csh in conv_shape: 
                    run_num += 1
                    with tf.compat.v1.Session(config=config) as sess:
                        
                        NAME = f"{cl}xConv({cs:>03})_shape{csh}-{dl}xDense({ds:>03})-{time.time():10.0f}"
                        

                        tensorboard = TensorBoard(log_dir=f'logs-optimize/{NAME}')
                        model = None
                        model = tf.keras.models.Sequential()

                        model.add(Conv2D(cs, (csh, csh), activation='relu', input_shape=pic_shape))
                        model.add(MaxPooling2D())

                        for i in range(cl-1):
                            model.add(Conv2D(cs, (csh, csh), activation='relu'))
                            model.add(MaxPooling2D())

                        model.add(Flatten())

                        for x in range(dl):
                            model.add(Dense(ds, activation='relu'))

                        model.add(Dense(label_count, activation='softmax'))

                        model.compile(optimizer='adam',
                                      loss='sparse_categorical_crossentropy',
                                      metrics=['accuracy'])

                        history = model.fit(X_training, y_training, 
                                            batch_size=25, epochs=10,
                                            validation_data=(X_test, y_test),                                   
                                            callbacks=[tensorboard, stop])

                        loss = history.history['loss']
                        accuracy = history.history['accuracy']

                        val_loss = history.history['val_loss']
                        val_accuracy = history.history['val_accuracy']

                        print(f"{(run_num/total)*100:<5.1f}% - {NAME} Results: ")
    #                     print(f"Test Accuracy: {val_accuracy[-1]:>2.4f}")
    #                     print(f"Test loss: {val_loss[-1]:>2.4f}")
                    